In [1]:
%config Completer.use_jedi = False

import h5py
import numpy as np
import json

In [2]:
file = h5py.File('/Users/lucamasserano/Desktop/uni/cmu/ada/ada_code/data/1.hdf5', 'r')
file.keys()

<KeysViewHDF5 ['E', 'hl_x', 'meta', 'y']>

## True energy

In [3]:
true_nrg = file['y']
true_nrg

<HDF5 dataset "y": shape (1200,), type "<f4">

In [4]:
true_nrg[0]

655.69965

## High level features (obsolete version, now done in C++)

In [5]:
hl_feats = file['hl_x']
hl_feats

<HDF5 dataset "hl_x": shape (1200, 7), type "<f4">

In [6]:
hl_feats[0]

array([8.8310101e+02, 9.5349579e+02, 5.2524761e+03, 1.6910354e+04,
       3.8697146e+03, 5.7824880e-01, 7.8367944e+00], dtype=float32)

## Muons hits

Dimensions: (muon, calorimeter, z-cell, x-cell, y-cell). <br>
The dimension named 'calorimeter' apparently is just a *box* containing the actual grid 

In [7]:
hits = file['E']
hits

<HDF5 dataset "E": shape (1200, 1, 50, 32, 32), type "<f4">

#### Features V[0] and V[1]

In [8]:
hits[0][0].shape

(50, 32, 32)

In [9]:
np.where(hits[0][0] <= 0.1)

(array([ 0,  0,  0, ..., 49, 49, 49]),
 array([ 0,  0,  0, ..., 31, 31, 31]),
 array([ 0,  1,  2, ..., 29, 30, 31]))

In [10]:
np.sum(hits[0][0][np.where(hits[0][0] > 0.1)])# + np.sum(hits[0][0][np.where(hits[0][0] > 0.1)])

1.6827272

In [11]:
hits[()].squeeze().sum(axis=(1,2,3), where=(hits[()].squeeze() > 0.1)).shape

(1200,)

In [12]:
hits[()].squeeze().squeeze().shape

(1200, 50, 32, 32)

In [13]:
list(range(len(hits.shape)))[-3:]

[2, 3, 4]

In [14]:
hits[0][0].sum(axis=(0,1,2))

7.836794

## Metadata

In [15]:
metadata = json.loads(file['meta'][()])
metadata

{'e_mean': 0.000518606039858202,
 'e_sum': 31863.155088887936,
 'e_square_sum': 188868.6728268921,
 'len': 1200,
 'hl_feats': ['sume2rz_0',
  'sume2rz_1',
  'sume2rz_2',
  'sume2rz_3',
  'sume2rz_4',
  'rmom',
  'esum'],
 'hl_mean': [6001.090446607977,
  7430.592034116686,
  8066.831258693412,
  7832.179394274016,
  7573.081590550582,
  0.7213336141548932,
  26.552629240739922],
 'hl_sum': [7201308.535929573,
  8916710.440940022,
  9680197.510432094,
  9398615.273128819,
  9087697.908660699,
  865.6003369858718,
  31863.155088887906],
 'hl_square_sum': [123383767717.06184,
  163792438507.3064,
  201351317940.6949,
  186933638557.56647,
  186802666204.0862,
  866.4390811842718,
  6586960.255155739]}

## Muon 16 features

In [16]:
import pandas as pd
import seaborn as sns
import pylab as plt
from tqdm import tqdm
import os

import sys
sys.path.append('../muon_regression/')

In [18]:
from scripts import prep_data

In [19]:
data = pd.read_csv('./data/MuEloss_N396000_em0.1.asc', 
                   sep=' ', header=None,
                   names=[f'v[{i}]' for i in range(0, 16)] + ['true_energy'],
                   dtype=np.float128)

In [20]:
data

,v[0],v[1],v[2],v[3],v[4],v[5],v[6],v[7],v[8],v[9],v[10],v[11],v[12],v[13],v[14],v[15],true_energy
0,1.68273,6.15407,0.578249,1.269930,123.0730,46.68700,53.8146,144.41300,154.2800,104.0630,1.0,5.0,0.970702,3.0,3.0,0.442111,655.700
1,0.22672,3.22143,1.506160,1.407130,93.6621,4.45421,62.9253,37.83110,144.4230,136.3770,1.0,2.0,0.226720,0.0,0.0,0.000000,136.523
2,4.75289,9.47245,1.025090,0.150712,205.4830,630.37700,187.4360,147.50700,188.6130,174.3480,1.0,6.0,1.688000,5.0,4.0,1.251760,702.982
3,18.28610,15.39850,0.589862,0.355303,177.0400,174.67700,172.9010,153.49300,156.0430,321.6490,1.0,33.0,14.621800,6.0,6.0,1.304280,1536.120
4,6.80569,9.03234,0.933928,0.546600,159.8110,6.22464,118.5770,149.95700,340.8290,95.4652,1.0,17.0,6.159160,2.0,2.0,0.502743,1062.760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395995,47.43600,0.00000,0.573455,0.573455,175.5570,132.90900,185.6510,184.41300,100.8120,225.8470,1.0,289.0,1.636290,662.0,902.0,2.233330,1002.870
395996,7.23727,0.00000,0.905068,0.905068,121.6890,62.08430,75.9965,258.74200,98.0651,137.1370,1.0,839.0,6.739440,10.0,400.0,0.017752,920.702
395997,9.98190,0.00000,0.877561,0.877561,186.7020,34.92040,194.0400,9.71442,224.1320,68.8687,1.0,40.0,0.067988,57.0,368.0,0.676978,696.027
395998,4.50558,0.00000,0.865484,0.865484,95.1507,65.58240,43.3061,134.84000,133.0310,45.1880,1.0,529.0,4.209100,12.0,268.0,0.016958,266.856


### APPARENTLY THEY LOSE FLOAT PRECISION WHEN WRITING TO CSV! 
<b> E.G.: true_nrg[0] = 655.7 != 655.69965
    
Or maybe they lose it when passing through the C++ macro
    --> UPDATE: it seems that the C++ macro is ok. The problem is in ROOT itself: when it reads directly a root file, it collapses precision. Try loading the macro in ROOT and showing the first entry. It is already 655.7 and not 655.69965

<b> On average, roughly every three observation two are equal

In [94]:
len(data.groupby('true_energy'))

257744

In [95]:
len(data)

396000

### ADD true_energy FROM HDF5 (WHICH HAS FULL PRECISION) 

#### Read all root files into hdf5 and extract all true energies

In [33]:
with open('./use_files.txt', 'r') as usefiles:
    for file in tqdm(list(sorted(usefiles.read().split('\n')[:-1]))):
        prep_data.singleroot_to_hdf5(in_file=file, out_file=f'./data/hdf5/{file[42:][:-5]}.hdf5', write=True)

100%|██████████| 330/330 [59:06<00:00, 10.75s/it] 


In [21]:
correct_energies = []
for filename in sorted(sorted(os.listdir('./data/hdf5/'))[1:], key=lambda el: int(el[:-5])):
    file_hdf5 = h5py.File(f'./data/hdf5/{filename}', 'r')
    correct_energies.append(file_hdf5['y'][:])

In [22]:
correct_energies = np.vstack(correct_energies).reshape(-1)

In [23]:
data['correct_true_energy'] = correct_energies

In [24]:
data.iloc[150000:151000,:]

,v[0],v[1],v[2],v[3],v[4],v[5],v[6],v[7],v[8],v[9],v[10],v[11],v[12],v[13],v[14],v[15],true_energy,correct_true_energy
150000,6.84788,0.0,0.889322,0.889322,125.105,8.47112,135.3200,161.056,116.8650,111.380,1.0,876.0,6.400980,14.0,345.0,0.020013,664.744,664.744324
150001,8.51662,0.0,0.223191,0.223191,135.577,68.03220,76.3752,181.892,144.8110,184.549,1.0,25.0,0.065307,84.0,349.0,0.621097,915.738,915.738403
150002,10.01820,0.0,0.718253,0.718253,141.403,72.01520,159.8880,142.407,99.3925,198.044,1.0,89.0,0.198149,202.0,509.0,0.712468,819.639,819.639282
150003,24.15480,0.0,0.869094,0.869094,157.541,168.60900,175.4710,149.807,184.9460,111.165,1.0,89.0,0.172297,167.0,724.0,0.202956,1161.770,1161.768188
150004,29.94690,0.0,0.783768,0.783768,179.765,35.17800,142.4740,192.991,151.6140,197.931,1.0,259.0,0.598028,468.0,656.0,3.381360,1475.870,1475.867310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150995,12.37290,0.0,0.232080,0.232080,159.833,89.92970,45.8028,193.364,135.9910,223.383,1.0,81.0,0.266184,347.0,580.0,1.913070,1822.450,1822.451904
150996,8.36316,0.0,0.808258,0.808258,127.131,147.08700,126.8060,142.648,58.1920,108.403,1.0,27.0,0.020092,15.0,414.0,0.021234,590.608,590.607727
150997,26.09350,0.0,0.404762,0.404762,155.140,141.21900,144.8430,165.170,176.6930,140.209,1.0,138.0,0.882840,638.0,877.0,4.721230,1827.710,1827.712036
150998,22.45530,0.0,0.973750,0.973750,173.460,73.50150,73.2588,212.434,97.2388,292.527,1.0,141.0,0.263073,585.0,499.0,4.771330,955.221,955.220825


### There are still some duplicates!

In [29]:
len(data.correct_true_energy.unique())

392613

In [26]:
data[data.duplicated(subset='correct_true_energy', keep=False)].sort_values(by='correct_true_energy', ascending=True)

,v[0],v[1],v[2],v[3],v[4],v[5],v[6],v[7],v[8],v[9],v[10],v[11],v[12],v[13],v[14],v[15],true_energy,correct_true_energy
14282,0.28499,3.14892,1.850860,0.393191,52.6928,40.42330,39.5912,0.787409,103.7560,18.5160,1.0,1.0,0.151489,1.0,1.0,0.133501,129.963,129.963470
219775,3.71440,0.00000,2.002930,2.002930,85.5209,24.55350,24.0221,144.026000,52.3162,110.8110,1.0,299.0,3.479390,6.0,220.0,0.024965,129.963,129.963470
302843,5.18890,0.00000,0.731363,0.731363,117.9590,3.46006,155.4280,182.635000,21.2636,34.1255,1.0,601.0,4.922920,13.0,209.0,0.011896,138.718,138.718475
70374,6.16174,0.00000,0.455881,0.455881,100.1040,87.22480,85.8626,73.418600,43.3046,136.3620,1.0,733.0,5.898810,7.0,257.0,0.011714,138.718,138.718475
280070,4.21296,0.00000,1.279220,1.279220,81.3125,72.25420,158.1910,15.157600,20.8813,90.8563,1.0,441.0,3.987260,9.0,219.0,0.016223,139.460,139.459991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362565,26.67790,0.00000,0.471205,0.471205,172.3270,53.84450,109.4230,166.413000,187.8790,24.4496,1.0,211.0,0.972798,499.0,509.0,1.864850,1995.560,1995.559082
298251,16.94740,0.00000,0.283213,0.283213,148.3340,129.95000,170.1700,72.453400,122.1320,101.9280,1.0,124.0,0.311108,354.0,473.0,2.593070,1997.580,1997.579102
357938,64.53500,0.00000,0.370883,0.370883,190.1110,129.04200,195.1180,196.749000,260.4960,177.7240,1.0,322.0,1.602690,947.0,892.0,1.235820,1997.580,1997.579102
130401,21.87620,0.00000,0.439889,0.439889,194.5540,202.17200,229.1790,64.373600,131.7790,200.2690,1.0,116.0,0.236380,216.0,765.0,0.313064,1997.920,1997.917725


In [28]:
data.drop(columns='true_energy').to_csv('./data/MuEloss_correct.asc', index=False, sep=' ', header=False)

## Random things

In [ ]:
pwd

In [ ]:
data = pd.read_csv('./data/MuEloss_N396000_em0.1.asc', 
                   sep=" ", header=None,
                   names=[f'v[{i}]' for i in range(0, 16)] + ['true_energy'])

In [ ]:
s = set(range(data.shape[0]))

In [ ]:
len(s)

In [ ]:
some_idxs = np.random.choice(data.index, size=1000)
some_idxs.shape

In [ ]:
s.difference_update(set(some_idxs))

In [ ]:
len(s)

In [ ]:
data.to_numpy()[list(({}))]

In [ ]:
data_arr = data.to_numpy()

In [ ]:
data_arr[np.isin(data_arr[:, -1], data_arr[:, -1][:15])][:,-1]

In [ ]:
np.where(np.isin(data_arr[:,-1], 1957.78))[0]